In [1]:
from langchain_ollama import ChatOllama
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import time

# 모델 일관성을 위한 상수 정의
MODEL_NAME = "qwen3:1.7b"
BASE_URL = "http://localhost:11434"
EMBEDDING_MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"

In [2]:

from langchain_ollama import OllamaEmbeddings


print("==> 1. 문서 로딩 → PDF 읽기...")
loader = PyPDFLoader('../data/tutorial-korean.pdf')
documents = loader.load()
print(f"  총 {len(documents)}페이지 로드 완료")

print("==> 2. 문서 분할 → 작은 청크로 나누기")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,     # 조금 더 작게
    chunk_overlap=150,  # 중복 감소
)

chunks = text_splitter.split_documents(documents)
print(f"  {len(chunks)}개 청크 생성 완료")

print("==> 3. 벡터화 → 임베딩으로 변환")
embeddings = HuggingFaceEmbeddings(
        model_name=EMBEDDING_MODEL_NAME,
        model_kwargs={'device': 'cpu'},
        encode_kwargs={'normalize_embeddings': True}
    )

print("==> 4. 저장 → FAISS 벡터스토어에 저장")
vectorstore = FAISS.from_documents(chunks, embeddings)

print("===> 5. 검색 → 질문과 유사한 문서 찾기")
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 4}  # 6 → 4로 감소
)


print("===> 6. 생성 → LLM으로 답변 생성")
llm = ChatOllama(
    model="qwen3:1.7b",
    base_url="http://localhost:11434",
    temperature=0.1,
    num_predict=1500  # max_tokens → num_predict
)



==> 1. 문서 로딩 → PDF 읽기...
  총 39페이지 로드 완료
==> 2. 문서 분할 → 작은 청크로 나누기
  76개 청크 생성 완료
==> 3. 벡터화 → 임베딩으로 변환


c:\Users\user\AppData\Local\pypoetry\Cache\virtualenvs\langchain-basic-AGfTwH54-py3.12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


==> 4. 저장 → FAISS 벡터스토어에 저장
===> 5. 검색 → 질문과 유사한 문서 찾기
===> 6. 생성 → LLM으로 답변 생성


In [3]:

# 한국어 최적화 프롬프트
prompt_template = """
당신은 BlueJ 프로그래밍 환경 전문가입니다. 
아래 문서 내용을 바탕으로 정확하고 친절한 답변을 제공해주세요.

문서 내용:
{retrieveDocuments}

질문: {question}

답변 규칙:
1. 문서 내용만을 근거로 답변하세요
2. 단계별 설명이 필요하면 순서대로 작성하세요  
3. 구체적인 메뉴명, 버튼명을 포함하세요
4. 문서에 없는 정보는 "문서에서 찾을 수 없습니다"라고 하세요

답변:"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["retrieveDocuments", "question"]
)
print(" 프롬프트 설정 완료")

# ===================================
# 7. QA 체인 생성
# ===================================
print("\n ===> 7.  QA 체인 생성...")
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt, "document_variable_name": "retrieveDocuments" },
    return_source_documents=True
)
print("  RAG 파이프라인 구축 완료!")

# ===================================
# 8. 테스트 질문들
# ===================================
test_questions = [
    "BlueJ에서 객체를 생성하는 방법은 무엇인가요?",
    "컴파일 오류가 발생했을 때 어떻게 확인할 수 있나요?", 
    "디버깅을 위해 중단점을 설정하는 방법을 알려주세요",
    "코드패드는 무엇이고 어떻게 사용하나요?",
    "애플릿을 만들고 실행하는 방법을 설명해주세요"
]

print("\n" + "=" * 60)
print(" RAG 시스템 테스트")
print("=" * 60)

# ===================================
# 9. 질문 및 답변 실행
# ===================================
for i, question in enumerate(test_questions, 1):
    print(f"\n【테스트 {i}/5】")
    print(f" 질문: {question}")
    print(" 답변 생성 중...")
    
    # RAG 실행
    result = qa_chain.invoke({"query": question})
    answer = result["result"]
    source_docs = result["source_documents"]
    
    print(f"\n 답변:")
    print("-" * 50)
    print(answer)
    
    # 참조 문서 정보
    print(f"\n 참조 문서:")
    for j, doc in enumerate(source_docs[:3], 1):
        page = doc.metadata.get('page', 'N/A')
        preview = doc.page_content[:80].replace('\n', ' ')
        print(f"   {j}. 페이지 {page}: {preview}...")
    
    print("\n" + "-" * 40)

 프롬프트 설정 완료

 ===> 7.  QA 체인 생성...
  RAG 파이프라인 구축 완료!

 RAG 시스템 테스트

【테스트 1/5】
 질문: BlueJ에서 객체를 생성하는 방법은 무엇인가요?
 답변 생성 중...

 답변:
--------------------------------------------------
<think>
Okay, let's see. The user is asking how to create an object in BlueJ. I need to answer based on the provided document.

First, I remember that in the document, there's a section about creating objects from library classes. The user mentioned the "Tools - Use Libraries Class" menu. So, the steps would involve selecting that menu, entering the class name, and then creating the object.

Wait, the document says that you need to select the menu, input the class name, and then the list of constructors and static methods appears. The user can choose one of them. Also, the example given is using ArrayList or String classes. So, the answer should outline the steps: open the menu, enter the class name, and select the constructor or method.

I need to make sure I mention the exact menu name and the steps clearl